# Instalamos e importamos librerías

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 218.4MB 60kB/s 
     |████████████████████████████████| 204kB 44.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814406 sha256=3a5ebb2e53e199dcd3a94ac11fee12c4363bab36c13f2ba84ab7a3fe648013b6
  Stored in directory: /root/.cache/pip/wheels/1e/5e/6a/17e906c94ec7246f260330a66e44a06a0809033ba2738a74a8
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 32 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# 2016 2C 1 Parcial

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [0]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30),
]

In [0]:
documents = sc.parallelize(documents)
scores = sc.parallelize(scores)

In [0]:
terms = documents.flatMap(lambda x: [(word, x[0]) for word in x[1].split()])

In [0]:
scores_by_word = scores.map(lambda x: (x[1], (x[0], x[2])))

In [61]:
total = terms.join(scores_by_word)
total.collect()

[('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('computer', (3, ('martin', 30)))]

In [88]:
by_user = total.map(lambda x: ((x[1][1][0], x[1][0]), x[1][1][1])).reduceByKey(lambda x, y: x + y).cache()
by_user.first()

(('thom', 1), 2)

In [96]:
users = scoresRDD.map(lambda x: x[0]).distinct() # Usuarios unicos
docs = documentsRDD.map(lambda x: x[0])
users_docs = users.cartesian(docs).map(lambda x: ((x[0],x[1]), 0)).cache()
users_docs.collect()

[(('thom', 1), 0),
 (('thom', 2), 0),
 (('thom', 3), 0),
 (('thom', 4), 0),
 (('martin', 1), 0),
 (('martin', 2), 0),
 (('martin', 3), 0),
 (('martin', 4), 0),
 (('thom', 5), 0),
 (('thom', 6), 0),
 (('thom', 7), 0),
 (('thom', 8), 0),
 (('thom', 9), 0),
 (('martin', 5), 0),
 (('martin', 6), 0),
 (('martin', 7), 0),
 (('martin', 8), 0),
 (('martin', 9), 0)]

In [98]:
by_user.rightOuterJoin(users_docs).map(lambda x: (x[0][0], x[0][1], 0 if x[1][0] is None else x[1][0])).collect()

[('martin', 2, 0),
 ('thom', 6, 0),
 ('thom', 1, 2),
 ('thom', 9, 0),
 ('martin', 5, 0),
 ('martin', 4, 0),
 ('thom', 8, 0),
 ('thom', 3, 0),
 ('martin', 7, 0),
 ('thom', 2, 0),
 ('martin', 6, 0),
 ('martin', 1, -2),
 ('thom', 5, 0),
 ('martin', 9, 0),
 ('thom', 4, 0),
 ('martin', 8, 0),
 ('martin', 3, 60),
 ('thom', 7, 0)]

# 2017 1C 1 Parcial

Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

In [0]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

stats = sc.parallelize(players_all_time_stats)
scores = sc.parallelize(scores)

In [123]:
stats = stats.map(lambda x: (x[0], (x[1],x[2])))
stats.first()

(1, ('Manu Ginobili', 800))

In [115]:
scores_by_match = scores.map(lambda x: ((x[0], x[1]), x[3])).reduceByKey(lambda x, y: x + y)\
                                  .map(lambda x: (x[0][0], x[1]))
scores_by_match.first()

(1, 200)

In [0]:
resul = scores_by_match.join(stats).filter(lambda x: x[1][0] > x[1][1][1]).map(lambda x: (x[1][1][0])).distinct()

In [129]:
resul.collect()

['Kobe Bryant', 'Marc Gasol']